In [ ]:
import astropy
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import os
from pathlib import Path
from glob import glob
import re
import logging
from rich.progress import Progress
import numpy as np
import sunpy
import pandas as pd
from datetime import datetime, date, timedelta
import sscws
from sunpy.net import Fido, attrs as a
from functools import reduce

In [ ]:
fname_HAO = []#glob("fdl_space_weather_data/HAO_data/data_fits/*/*.fits")
fname_stereo = glob("ssa_live_twin_development_data/*/*.fts")

fnames = fname_HAO + fname_stereo
print(len(fnames))

In [ ]:
broken_files = []
with Progress() as progress:
    task = progress.add_task("Plotting Stereo Data", total = len(fnames))
    for fname in fnames:
        plt.close("all")
        path_components_fname = fname.split("/")[:-1]
        path_components = ["plots"]
        path_components = path_components + path_components_fname
        joined_path = os.path.join(*path_components)
        if not os.path.exists(joined_path):
            os.makedirs(joined_path)
        try:
            plotname = joined_path +"/beacon_substitute_"+ fname.split("/")[-1].split(".")[0]+".jpg"
            img_data = fits.getdata(fname)
            plt.imshow(img_data,cmap = "gray")
            plt.title("Beacon Substitute: "+fname.split("/")[-1].split(".")[0])
            plt.savefig(plotname)
        except Exception as e:
            logging.warning("Error: {} - File {}! Check manually".format(e,fname))
            broken_files.append(fname)
        progress.update(task,advance = 1)


In [ ]:
fname_part =fname.split("/")[-1].split(".")[0]

In [ ]:
png_name = 'plots/ssa_live_twin_development_data/cor2/secchi_l0_b_img_cor2_20120307_20120307_022400_d4c2b.png'

In [ ]:
png_jpg_size_ratio = os.stat(png_name).st_size/os.stat(plotname).st_size 
# Irrelevant, going up against the ICER algorithm for beacon data...

## Set up Satellite Position Data

In [ ]:
# Ephemeris Files for wanted objects
l5_positions_fname = "L5_positions_ephemeris.txt"
stereoA_positions_fname = "StereoA_positions_ephemeris.txt"
stereoB_positions_fname = "StereoB_positions_ephemeris.txt"
SOHO_positions_fname = "SOHO_positions_ephemeris.txt"

In [ ]:
def load_ephemeris_data(fname:str) -> list:
    '''
        Loads data from horizon ephemeris files, passed as argument.
    '''
    with open(fname) as f:
        data = f.readlines()

    start_position = 0
    end_position = 0
    for i, line in enumerate(data):
        if "$$SOE" in line:
            start_position = i + 1
        if "$$EOE" in line:
            end_position = i
            
    return data[start_position:end_position]

In [ ]:
l5_positions_data = load_ephemeris_data(l5_positions_fname)
stereoA_positions_data = load_ephemeris_data(stereoA_positions_fname)
stereoB_positions_data = load_ephemeris_data(stereoB_positions_fname)
SOHO_positions_data = load_ephemeris_data(SOHO_positions_fname)

In [ ]:


def data_to_vectors(data,Time:list = None):
    '''
        Takes the data returned by the load_data function and turns it into the required vectors
    '''
    time = Time if Time is not None else []
    x = []
    y = []
    z = []
    r = []
    
    for line in data:
        t,p = line.strip("\n").split("     ")
        t2 = t.split(" ")[1] + " " + t.split(" ")[2]
        if Time is not None:
            assert datetime.strptime(sa_time2,"%Y-%b-%d %H:%M") in Time,"New Time found in line {}".format(line)
        else:
            time.append(datetime.strptime(t2,"%Y-%b-%d %H:%M"))
        space_split = [i for i in p.split(" ") if i != ""]
        RA = float(space_split[0])*np.pi/180 # Radian for numpy
        DEC = float(space_split[1])*np.pi/180 # Radian for numpy
        
        cr = float(space_split[2]) # in km
        #dont use rdot here
        cx = cr*np.cos(DEC)*np.cos(RA*np.cos(DEC)) # km
        cy = cr*np.cos(DEC)*np.sin(RA*np.cos(DEC)) # km
        cz = cr*np.sin(DEC) # km
        
        x.append(cx)
        y.append(cy)
        z.append(cz)
        r.append(cr)
    return time, x,y,z,r
        
Time,L5x,L5y,L5z,L5r = data_to_vectors(l5_positions_data)
_,SAx,SAy,SAz,SAr = data_to_vectors(stereoA_positions_data,Time)
_,SBx,SBy,SBz,SBr = data_to_vectors(stereoB_positions_data,Time)
_,Sohox,Sohoy,Sohoz,Sohor = data_to_vectors(SOHO_positions_data, Time)


In [ ]:
initial_colnames = ["L5 x [km]","L5 y [km]","L5 z [km]","L5 r [km]","SA x [km]","SA y [km]","SA z [km]","SA r [km]","SB x [km]","SB y [km]","SB z [km]","SB r [km]", "SOHO x [km]","SOHO y [km]","SOHO z [km]", "SOHO r [km]"]
initial_data = (np.asarray([L5x,L5y,L5z,L5r,SAx,SAy,SAz,SAr,SBx,SBy,SBz,SBr,Sohox,Sohoy,Sohoz,Sohor])).T

df = pd.DataFrame(initial_data,index = Time, columns = initial_colnames)
df.head()
df["Distance L5 Stereo A [km]"] = np.sqrt((df["L5 x [km]"] - df["SA x [km]"])**2+(df["L5 y [km]"] - df["SA y [km]"])**2+(df["L5 z [km]"] - df["SA z [km]"])**2)
df["Distance L5 Stereo B [km]"] = np.sqrt((df["L5 x [km]"] - df["SB x [km]"])**2+(df["L5 y [km]"] - df["SB y [km]"])**2+(df["L5 z [km]"] - df["SB z [km]"])**2)                                           
df["Stereo AB Angle [deg]"] = np.arccos((df["SA x [km]"]*df["SB x [km]"]+df["SA y [km]"]*df["SB y [km]"]+df["SA z [km]"]*df["SB z [km]"])/(df["SA r [km]"]*df["SB r [km]"]))*180/np.pi                                          
df["Stereo A Soho Angle [deg]"] = np.arccos((df["SA x [km]"]*df["SOHO x [km]"]+df["SA y [km]"]*df["SOHO y [km]"]+df["SA z [km]"]*df["SOHO z [km]"])/(df["SA r [km]"]*df["SOHO r [km]"]))*180/np.pi                          
df["Stereo B Soho Angle [deg]"] = np.arccos((df["SB x [km]"]*df["SOHO x [km]"]+df["SB y [km]"]*df["SOHO y [km]"]+df["SB z [km]"]*df["SOHO z [km]"])/(df["SB r [km]"]*df["SOHO r [km]"]))*180/np.pi                                              

# Statistics
df.describe()
                                          

### Create Plots from dataframe

In [ ]:
# Parameters
earth_l5_angle_degrees = 60
error_range_degrees = 5
required_distance_to_l5_km = 50000000 #km
max_angle_between_crafts_deg = earth_l5_angle_degrees + error_range_degrees
min_angle_between_crafts_deg = earth_l5_angle_degrees - error_range_degrees

In [ ]:


approx_date_last_B_contact = date(2016,9,1) #some time September 2016

ax = df.plot(title = "Distance of Stereo A to L5", xlabel = "Year", ylabel = "Distance [km]", y = "Distance L5 Stereo A [km]")
ax.axhline(y = required_distance_to_l5_km, linestyle = "-", color = "g", linewidth = 2)
plt.savefig("dist_a_l5.png")

ax = df.plot(title = "Distance of Stereo B to L5", xlabel = "Year", ylabel = "Distance [km]",y = "Distance L5 Stereo B [km]")
ax.axvline(x = approx_date_last_B_contact, linestyle = "-", color = "r", linewidth = 2)
ax.axhline(y = required_distance_to_l5_km, linestyle = "-", color = "g", linewidth = 2)
plt.savefig("dist_b_l5.png")


ax = df.plot(title = "Angle between Stereo A and Stereo B",xlabel = "Year", ylabel = "Angle [°]", y = "Stereo AB Angle [deg]",linestyle = ":")
ax.axhline(y = earth_l5_angle_degrees,linestyle = "--", color = "g", linewidth = 2)
ax.axhline(y = earth_l5_angle_degrees + error_range_degrees,linestyle = "--", color = "g", linewidth = 1, alpha = 0.5)
ax.axhline(y = earth_l5_angle_degrees - error_range_degrees,linestyle = "--", color = "g", linewidth = 1, alpha = 0.5)
ax.axvline(x = approx_date_last_B_contact, linestyle = "-", color = "r", linewidth = 2) 
plt.savefig("Angle_AB.png")

ax = df.plot(title = "Angle between Stereo A and SOHO",xlabel = "Year", ylabel = "Angle [°]", y = "Stereo A Soho Angle [deg]",linestyle = ":")
ax.axhline(y = earth_l5_angle_degrees,linestyle = "--", color = "g", linewidth = 2)
ax.axhline(y = earth_l5_angle_degrees + error_range_degrees,linestyle = "--", color = "g", linewidth = 1, alpha = 0.5)
ax.axhline(y = earth_l5_angle_degrees - error_range_degrees,linestyle = "--", color = "g", linewidth = 1, alpha = 0.5)
ax.axvline(x = approx_date_last_B_contact, linestyle = "-", color = "r", linewidth = 2) 
plt.savefig("Angle_ASOHO.png")

ax = df.plot(title = "Angle between Stereo B and SOHO",xlabel = "Year", ylabel = "Angle [°]", y = "Stereo B Soho Angle [deg]",linestyle = ":")
ax.axhline(y = earth_l5_angle_degrees,linestyle = "--", color = "g", linewidth = 2)
ax.axhline(y = earth_l5_angle_degrees + error_range_degrees,linestyle = "--", color = "g", linewidth = 1, alpha = 0.5)
ax.axhline(y = earth_l5_angle_degrees - error_range_degrees,linestyle = "--", color = "g", linewidth = 1, alpha = 0.5)
ax.axvline(x = approx_date_last_B_contact, linestyle = "-", color = "r", linewidth = 2) 
plt.savefig("Angle_BSOHO.png")



# Discover relevant periods based on data and L5 proximity

In [ ]:
# use query
df_angles_fit = df.query("`Stereo AB Angle [deg]` >= {} & `Stereo AB Angle [deg]` <= {}".format(min_angle_between_crafts_deg,max_angle_between_crafts_deg))
df_angles_perfect_fit = df.query("`Stereo AB Angle [deg]`== {}".format(earth_l5_angle_degrees))
df_stereoA_angles_fit = df.query("`Stereo A Soho Angle [deg]` >= {} & `Stereo A Soho Angle [deg]` <= {}".format(min_angle_between_crafts_deg,max_angle_between_crafts_deg))
df_stereoB_angles_fit = df.query("`Stereo B Soho Angle [deg]` >= {} & `Stereo B Soho Angle [deg]` <= {}".format(min_angle_between_crafts_deg,max_angle_between_crafts_deg))

df_StereoB_close = df.query("`Distance L5 Stereo B [km]` <= {}".format(required_distance_to_l5_km))
df_StereoA_close = df.query("`Distance L5 Stereo A [km]` <= {}".format(required_distance_to_l5_km))

# get times from df_{}.index
# Main point: Do angles fit? - more important than distance for now.
angle_AB_index = df_angles_fit.index
angle_ASOHO_index = df_stereoA_angles_fit.index
angle_BSOHO_index = df_stereoB_angles_fit.index
# Overlap?
overlap_ABASOHO = np.intersect1d(angle_AB_index,angle_ASOHO_index) #empty
overlap_ABBSOHO = np.intersect1d(angle_AB_index,angle_BSOHO_index) #empty
overlap_ASOHOBSOHO = np.intersect1d(angle_ASOHO_index,angle_BSOHO_index) 
# 24% of times, both angles A-SUN-SOHO, B-SUN-SOHO are within 55 to 65 degrees
# AB and Soho are never in a 60° pairing - ie no scenario of 60° on both angles between A,B and SOHO from either one
# in other words, never happens when either A, B are close to earth
union_index = reduce(np.union1d,(angle_AB_index, angle_ASOHO_index, angle_BSOHO_index))
#Need to recover consecutive groups - ie, each group only consists of elements that are no more than 1 day removed from each other.
time_differences = union_index[1:] - union_index[:-1]

In [ ]:
endpoints = [-1,len(union_index)]
for time_diff_jumps_idx,td in enumerate(time_differences):
    if td != 86400000000000: # = 1 day in ns
        endpoints.append(time_diff_jumps_idx)
endpoints = np.sort(endpoints)
#print(endpoints)
timeseries_batches = [union_index[endpoints[i-1]+1:endpoints[i]] for i in range(1,len(endpoints))] 

# Query Data from FIDO

In [ ]:
event_type = "CE"
# Can download SECCHI data with help from FIDO
event_batches = []
instrument_batches = []
for batch in timeseries_batches:
    min_time = np.min(batch)
    max_time = np.max(batch)
    time_requested_for_batch = a.Time(str(min_time),str(max_time))
    resulting_events_for_batch = Fido.search(time_requested_for_batch, a.hek.EventType(event_type))
    resulting_images_for_batch = Fido.search(time_requested_for_batch, a.Instrument("SECCHI"), a.Detector("COR1"))
    instrument_batches.append(resulting_images_for_batch)
    event_batches.append(resulting_events_for_batch)
'''
for batch in timeseries_batches:
    min_time = np.min(batch)
    max_time = np.max(batch)
    time_requested_for_batch = a.Time(str(min_time),str(max_time))
    resulting_events_for_batch = Fido.search(time_requested_for_batch, a.hek.EventType(event_type))
    resulting_images_for_batch = Fido.search(time_requested_for_batch, a.Instrument.secchi, a.Level.one)
    instrument_batches.append(resulting_images_for_batch)
    event_batches.append(resulting_events_for_batch)
'''

#a.Level restricts query to 0 images, regardless of Level set - unsure which level the image data is from


In [ ]:
instrument_batches

In [ ]:
for i in range(len(event_batches)):
    t_batch = timeseries_batches[i]
    min_time = np.min(t_batch)
    max_time = np.max(t_batch)
    n_events = len(event_batches[i]["hek"])
    print("Between {} and {}, there have been {} events logged.".format(min_time, max_time, n_events))

In [ ]:
from sscws import sscws

In [ ]:
sscws = sscws.SscWs()

In [ ]:
time = ["2022/06/01","2023/07/13"]
locations = sscws.get_locations(["soho","sun","stereoa","stereob"], time)

In [ ]:
soho_triple = [locations["Data"][0]["Coordinates"][0]["X"],locations["Data"][0]["Coordinates"][0]["Y"],locations["Data"][0]["Coordinates"][0]["Z"]]
sun_triple = [locations["Data"][1]["Coordinates"][0]["X"],locations["Data"][1]["Coordinates"][0]["Y"],locations["Data"][1]["Coordinates"][0]["Z"]]
stereoa_triple = [locations["Data"][2]["Coordinates"][0]["X"],locations["Data"][2]["Coordinates"][0]["Y"],locations["Data"][2]["Coordinates"][0]["Z"]]
stereob_triple = [locations["Data"][3]["Coordinates"][0]["X"],locations["Data"][3]["Coordinates"][0]["Y"],locations["Data"][3]["Coordinates"][0]["Z"]]


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=90, azim=0) 

sohox,sohoy,sohoz = [df["SOHO x [km]"],df["SOHO y [km]"],df["SOHO z [km]"]]
sunx,suny,sunz = [np.zeros_like(sohox),np.zeros_like(sohox),np.zeros_like(sohox)] #didnt query data, assume 0
stereoax,stereoay,stereoaz = [df["SA x [km]"],df["SA y [km]"],df["SA z [km]"]]
stereobx,stereoby,stereobz = [df["SB x [km]"],df["SB y [km]"],df["SB z [km]"]]

#sohoz = np.zeros_like(stereoax)
#sunz = np.zeros_like(sunx)
#stereoaz = np.zeros_like(stereoax)
#stereobz = np.zeros_like(stereobx)

ax.plot(sohox,sohoy,sohoz, label = "SOHO")
ax.plot(sunx,suny,sunz, label = "Sun")
ax.plot(stereoax,stereoay,stereoaz, label = "Stereo A")
ax.plot(stereobx,stereoby,stereobz, label = "Stereo B")

l_sohox = sohox[-1]
l_sohoy = sohoy[-1]
l_sohoz = sohoz[-1]
l_sunx = sunx[-1]
l_suny = suny[-1]
l_sunz = sunz[-1]
l_stereoax = stereoax[-1]
l_stereoay = stereoay[-1]
l_stereoaz = stereoaz[-1]
l_stereobx = stereobx[-1]
l_stereoby = stereoby[-1]
l_stereobz = stereobz[-1]

l5_pos = [df["L5 x [km]"][-1],df["L5 y [km]"][-1],df["L5 z [km]"][-1]]

ax.text(l_sohox,l_sohoy,l_sohoz, 'SOHO', fontsize=12, ha='center', va='bottom')
ax.text(l_sunx,l_suny,l_sunz, 'Sun', fontsize=12, ha='center', va='bottom')
ax.text(l_stereoax,l_stereoay,l_stereoaz, 'Stereo A', fontsize=12, ha='center', va='bottom')
ax.text(l_stereobx,l_stereoby,l_stereobz, 'Stereo B', fontsize=12, ha='center', va='bottom')
ax.text(*l5_pos, "L5",fontsize = 12, ha = "center",va = "bottom")

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
ax.axis("off")
plt.savefig("soho_sun_l5_stereo.png")

In [ ]:
df.tail()